<a href="https://colab.research.google.com/github/SonjaWangJQ/Keyboard-ML/blob/main/Research_On_Keyboard_Acoustic_Eavesdropping_and_Typing_Speed_in_Skype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The audio data was recorded and saved in WAV format with the following specifications:

Sampling Frequency: 44.1kHz
Bit Depth: 32-bit PCM
Audio Channel: Mono

In [ ]:

# turn mp4 into wav and sr = 44100
!pip3 install pydub
from pydub import AudioSegment
def mp4_to_wav(audio_file, output_file,target_sr = 16000):
  mp4_file1 = AudioSegment.from_file(audio_file, "mp4")
  wav_file1 = mp4_file1.export(output_file, format="wav")
  y, sr = librosa.load(wav_file1)

  y_resampled = librosa.resample(y, orig_sr=sr, target_sr = target_sr)

  sf.write(output_file, y_resampled, samplerate= target_sr)


In [ ]:

# keep audio as mono signal

def wav_read(audio_file):
    """Return 1D NumPy array of wave-formatted audio data denoted by filename.

    Input should be a string containing the path to a wave-formatted audio file.
    """
    sample_rate, signal = wav.read(audio_file)
    print('SAMPLE_RATE:',sample_rate)
    if type(signal[0]) == np.ndarray:
        return signal[:, 0]
    else:
        return signal


Data Proprocessing:
1. noise removal
2. keystrokes segmentation

In [ ]:

def frequency_filter(audio_file, output_file, sample_rate, low_freq, high_freq):
    """
    Filter out frequencies outside the specified range [low_freq, high_freq].

    Parameters:
        signal (np.ndarray): Input signal in the time domain.
        sample_rate (int): Sampling rate of the signal in Hz.
        low_freq (int): Lower cutoff frequency in Hz. Frequencies below this value will be removed.
        high_freq (int): Upper cutoff frequency in Hz. Frequencies above this value will be removed.

    Returns:
        np.ndarray: Filtered signal in the time domain.
    """
    signal = wav_read(audio_file)
    n = len(signal)
    freqs = np.fft.fftfreq(n, 1 / sample_rate)
    fft_signal = np.fft.fft(signal)

    # Find the indices of frequencies outside the specified range
    low_index = np.argmin(np.abs(freqs - low_freq))
    high_index = np.argmin(np.abs(freqs - high_freq))

    # Set frequencies outside the range to zero
    fft_signal[:low_index] = 0
    fft_signal[high_index:] = 0

    # Apply inverse FFT to obtain the filtered signal in time domain
    # filtered_signal = np.fft.ifft(fft_signal)
    filtered_signal = np.fft.ifft(fft_signal)

    # Take the real part of the filtered signal (ignore imaginary parts due to numerical errors)
    filtered_signal = np.real(filtered_signal)
    wav.write(output_file, sample_rate, filtered_signal.astype(np.int16))




In [ ]:

### abs(signal)
def calculate_energy(signal):
    return abs(signal)
def calculate_A_energy(signal,sample_rate,window_size = 10):

    energy = calculate_energy(signal)
    A_energy2 = np.zeros(len(signal))

    for i in range(len(signal) - window_size + 1):
        A_energy2[i] = np.sum(energy[i:i+window_size])

    return A_energy2


def A_t(signal,sample_rate,t): # t = 0.1
  W = int(sample_rate* t)
  print('A_t_Winowsize:',W)
  A_energy = calculate_A_energy(signal,sample_rate = 44100,window_size = 10)
  print('Max of Amptitude:',max(A_energy))

  A_t = []
  n = 0
  if max(A_energy) <= 10000:
    # threshold = 1000
    threshold = 0.2 * int(max(A_energy))
    while n <= (len(signal) - W + 1) :
      if (A_energy[n]) >= threshold and (A_energy[n] >= max(A_energy[n: n+W])):
        # A_t.append([n,A_energy[n]])
        A_t.append(n)
        n += W
      else:
        n += 1

  else:
    # threshold = 2000
    threshold = 0.5 * int(max(A_energy))
    while n <= (len(signal) - W + 1) :
      if (A_energy[n]) >= threshold and (A_energy[n] >= max(A_energy[n: n+W])):
        # A_t.append([n,A_energy[n]])
        A_t.append(n)
        n += W
      else:
        n += 1
  return A_t



In [ ]:
def keystrokes(audio_file,t,sample_rate =44100):
    signal = wav_read(audio_file)
    A_energy = calculate_A_energy(signal,sample_rate ,window_size = 10)
    # At =   A_t(signal,threshold,t,sample_rate= 44100)
    At = A_t(signal,sample_rate,t)
    a = int(0.2*t* sample_rate)
    b = int(0.8*t* sample_rate)
    # print(a,b)

    keystrokes = []


    for i in At:

      if (i+b)<= len(signal):
        keystroke = signal[i-a: i+b]
        keystrokes.append(keystroke)
        # print(i)
      if (i + b) > len(signal):
        keystroke = signal[i-a :]
        keystrokes.append(keystroke)
        # print(i)

    return np.array(keystrokes)
    # print(len(keystrokes))

1. Data Augmentation pipelime
2. Feature Extraction pipeline


In [ ]:


class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------

  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)




  # time shift

  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

  # amplity all timedata
  @staticmethod
  def time_Amp(aud, Amp_size):
    sig,sr = aud
    Amp_sig = sig * Amp_size
    return (Amp_sig , sr)


  @staticmethod
  def compute_mfcc(aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=32):

            sig, sr = aud
            n_fft = int(sr * window_size)
            hop_length = int(sr * step_size)

            mfccs = librosa.feature.mfcc(
                y=sig.numpy(),
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_filters
            )
            return mfccs.T


  @staticmethod
  def compute_stft(aud, n_fft=2048, hop_length=512, window='hann'):
    sig, sr = aud
    audio = sig.squeeze().numpy()

    # Compute STFT
    stft_result = librosa.core.stft(audio, n_fft=2048, hop_length=512, window='hann')

    # Get magnitude spectrum
    magnitude = np.abs(stft_result)

    # Get frequency axis
    freqs = librosa.core.fft_frequencies(sr=sr, n_fft=2048)

    return freqs, magnitude


load data

In [ ]:

# ----------------------------
# SoundDS1 no ampf
# ----------------------------
class SoundDS(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    # self.shift_pct = -0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts


  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    Amp_aud = AudioUtil.time_Amp(aud,self.amp_size)
    mfcc_aud = AudioUtil.compute_mfcc(Amp_aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)
    # return mfcc_aud, class_id
    return mfcc_aud, class_id, audio_path # for mapping

Key Classification

In [ ]:
#SVM model
#
# Best parameters: {'C': 10, 'kernel': 'rbf'} 0.84
#Mfcc: aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=64
#  parameter grid for grid search
from sklearn.metrics import top_k_accuracy_score


param_grid = {
    'C': [1, 5, 10, 100],
    # 'C': [10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
    # 'gamma':[0.001]
    }

#  SVM model
svm = SVC()

#  grid search object with 10-fold cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

#  best model from grid search
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)
target_names = [f'Class {i}' for i in range(36)]
#   classification report
cr = classification_report(y_test, y_pred, target_names=target_names)
# N = 5  # Change this to the desired value
# top_n_accuracy = top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N)



N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N) for N in N_values]

# Print results
print("Best parameters:", grid_search.best_params_)
print("Classification Report:")
print(cr)
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")


In [ ]:


# Multinomial Logistic Regression  X,y 0.2 ；L2；10 fold across MFCC

#X,y 0.2 ；L2；10 fold across MFCC

multi_logreg = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter=1000)

param_grid = {
    'C': [ 1,10 ],
    'penalty': ['l2']
}

# 10 fold across
grid_search = GridSearchCV(multi_logreg, param_grid, cv=10, n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)



target_names = [f'Class {i}' for i in range(36)]

# report
cr = classification_report(y_test, y_pred, target_names=target_names)

# Top-N accuracy score
N_values = [1, 2, 3, 4, 5]  # You can add more N values if needed
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.predict_proba(X_test_scaled), k=N) for N in N_values]

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(cr)

In [ ]:
# random forests
import pydot
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report

# Assuming you have loaded X_train_scaled, y_train, X_test_scaled, and y_test

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [ 5, 10],
    'min_samples_leaf': [ 2, 4],
    'bootstrap': [True, False]
}



# Create and fit the Random Forest model using GridSearchCV
random_forest = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(random_forest, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_random_forest = grid_search.best_estimator_

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
y_test_pred_probs = cross_val_predict(best_random_forest, X_test_scaled, y_test, cv=kf, method='predict_proba')

# ... (Rest of your code for calculating top-N accuracy scores and generating the classification report)
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = []
for N in N_values:
    top_n_correct = 0
    for true_label, pred_probs in zip(y_test, y_test_pred_probs):
        top_n_indices = np.argsort(pred_probs)[::-1][:N]  # Indices of top-N classes
        if true_label in top_n_indices:
            top_n_correct += 1
    top_n_accuracy = top_n_correct / len(y_test)
    top_n_accuracies.append(top_n_accuracy)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Generate classification report
target_names = [f'Class {i}' for i in range(36)]
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
cr = classification_report(y_test, y_test_pred, target_names=target_names)
# Print classification report
print("Classification Report:")
print(cr)



Data Augmentation part
one example for data shift + 10ms

In [ ]:

# ----------------------------
# SoundDS +10ms
# ----------------------------
class SoundDS1(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename

    self.channel = 1
    self.shift_pct = 0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts





  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    shift_aud = AudioUtil.time_shift(aud, self.shift_pct)
    mfcc_aud = AudioUtil.compute_mfcc(shift_aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)

    return mfcc_aud, class_id, audio_path


In [ ]:
# shift  append into X,y
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift= SoundDS(csv_filename)
X = []
y = []
path = []


for idx in range(len(sound_dataset_shift)):
    mfcc_data, class_id ,audio_path = sound_dataset_shift[idx]
    X.append(mfcc_data)
    y.append(class_id)
    path.append(audio_path)



In [ ]:
X = np.array(X)
y = np.array(y)
path = np.array(path)
# check length
assert X.shape[0] == len(y), "Shape mismatch between X and y"
assert X.shape[0] == len(path), "Shape mismatch between X and path"
print(X.shape)
num_samples, num_frames, num_coefficients, num_channels = X.shape
MFCC_data_2d = X.reshape(num_samples, num_frames * num_coefficients)


X_shuffled, y_shuffled,path_shuffled = shuffle(MFCC_data_2d, y, path, random_state=42)
# X_shuffled, y_shuffled = shuffle(MFCC_data_2d, y, random_state=42)
X_train, X_test, y_train, y_test,path_train, path_test = train_test_split(X_shuffled, y_shuffled, path_shuffled, train_size=0.8, random_state=42,stratify = y_shuffled)

(1467, 41, 32, 1)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # keep it
# X_val_scaled = scaler.transform(X_val)
print(X_train_scaled.shape, X_test_scaled.shape)

(1173, 1312) (294, 1312)


### append new data shift into trainset

In [ ]:
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift2 = SoundDS1(csv_filename)
X2 = []
y2 = []

for idx in range(len(sound_dataset_shift2)):
    mfcc_data2, class_id2, audio_path = sound_dataset_shift2[idx]
    if audio_path2 in path_train:
        X2.append(mfcc_data2)
        y2.append(class_id2)


In [ ]:
X2 = np.array(X2)
y2 = np.array(y2)

assert X2.shape[0] == len(y2), "Shape mismatch between X and y"
num_samples, num_frames, num_coefficients, num_channels = X2.shape

new_X2train = X2.reshape(num_samples, num_frames * num_coefficients)
print(new_X2train.shape)

(2346, 1312)


In [ ]:

X2_shuffled, y2_shuffled = shuffle( new_X2train , y2, random_state=42)
new_X2_train_scaled = scaler.fit_transform(X2_shuffled)
print(new_X2_train_scaled.shape, y2_shuffled)


(2346, 1312) [11  5 15 ... 34  6 17]


In [ ]:
# append X_train, y_train
X_train_extended = np.vstack([X_train_scaled, new_X2_train_scaled])
y_train_extended = np.concatenate([y_train, y2_shuffled])